In [ ]:
import pandas as pd
import hashlib as hl
import os
import numpy as np

In [ ]:
#constants
directory = "C://Users//RAGS//Desktop//Dev_Env//test_notes"
out_file = "errors.txt"

In [ ]:
def hash_file(filename):
   """"This function returns the SHA-1 hash
   of the file passed into it"""

   # make a hash object
   h = hl.sha1()

   # open file for reading in binary mode
   with open(filename,'rb') as file:

       # loop till the end of the file
       chunk = 0
       while chunk != b'':
           # read only 1024 bytes at a time
           chunk = file.read(1024)
           h.update(chunk)

   # return the hex representation of digest
   return h.hexdigest()

In [ ]:
def list_files(dir_path):
    file_paths = []
    for root, directories, files in os.walk(dir_path):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
    return file_paths

In [ ]:
def create_hash_file(filename):
    files = list_files(directory)
    hash = []
    for i in files:
        hash.append(hash_file(i))
    #hash
    df = pd.DataFrame(files, columns=['filenames'])
    df['hash'] = hash
    df.to_csv(filename)

In [ ]:
#google imports
from __future__ import print_function

import os.path
import sys

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import google.auth
from googleapiclient.http import MediaFileUpload

In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']

def create_service():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        
    # create drive api client
    service = build('drive', 'v3', credentials=creds)
    return service

In [ ]:
def upload_file(service, filename):
    #service = create_service()
    try:

        file_metadata = {'name': filename}
        media = MediaFileUpload(filename)
        # pylint: disable=maybe-no-member
        file = service.files().create(body=file_metadata, media_body=media,
                                      fields='id').execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

In [ ]:
def print_files_in_drive(service):
    #service = create_service()
    try:
        # Call the Drive v3 API
        results = service.files().list(
            pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print('No files found.')
            return
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

In [ ]:
def create_folder(service, folder_name):
    """ Create a folder and prints the folder ID
    Returns : Folder Id """

    # create drive api client
    #service = create_service()
    try:
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }

        # pylint: disable=maybe-no-member
        file = service.files().create(body=file_metadata, fields='id').execute()
                                      
        print(F'Folder ID: "{file.get("id")}".')
        return file.get('id')

    except HttpError as error:
        print(F'An error occurred: {error}')
        return None

In [ ]:
def upload_to_folder(service, folder_id, filename):
    """Upload a file to the specified folder and prints file ID, folder ID
    Args: Id of the folder
    Returns: ID of the file uploaded
    """
    try:
        file_metadata = {
            'name': filename,
            'parents': [folder_id]
        }
        media = MediaFileUpload(filename, resumable=True)
        # pylint: disable=maybe-no-member
        file = service.files().create(body=file_metadata, media_body=media,
                                      fields='id').execute()
        print(F'File ID: "{file.get("id")}".')
        return file.get('id')

    except HttpError as error:
        print(F'An error occurred: {error}')
        return None

In [ ]:
create_hash_file('old_hash.csv')

In [ ]:
create_hash_file('new_hash.csv')

In [ ]:
df1 = pd.read_csv('old_hash.csv')
df2 = pd.read_csv('new_hash.csv')
df=pd.concat([df1,df2])
df.drop(0,axis=1,inplace=True)
df
#df.to_excel('final.xlsx')
# df.drop_duplicates(keep=False, inplace=True)
# df

In [ ]:
service = create_service()

In [ ]:
print_files_in_drive(service)

In [ ]:
files=list_files(directory)
for file in files:
    upload_file(service, file)

In [ ]:
print('This message will be displayed on the screen.')

original_stdout = sys.stdout # Save a reference to the original standard output

with open(out_file, 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print('This message will be written to a file.')
    sys.stdout = original_stdout # Reset the standard output to its original value